In [171]:
import tensorflow as tf
import xlrd

In [172]:
import pandas as pd

In [173]:
d = pd.read_table('dataset.txt')

In [174]:
b = pd.read_table('testset.txt')

In [175]:
import numpy as np

In [176]:
temp = d.values[:,0]
temp1 = b.values[:,0]

In [177]:
temp = np.reshape(temp, (-1,1))
temp1 = np.reshape(temp1, (-1,1))
a = np.hstack((temp, d.values[:,2:]))
t = np.hstack((temp1, b.values[:, 2:]))

In [178]:
d.head()

,year,player,age,g,mp,per,ts,3par,ftr,orb,...,ows,dws,ws,ws_48,obpm,dbpm,bpm,vorp,game_win,mvp_label
0,1986,Dominique Wilkins,26,78,3049,23.3,0.536,0.037,0.372,9.5,...,7.0,3.9,10.8,0.170,3.5,0.0,3.5,4.3,50,0
1,1986,Randy Wittman,26,81,2760,13.0,0.555,0.018,0.153,2.0,...,3.9,2.0,5.9,0.103,0.7,-0.1,0.6,1.8,50,0
2,1986,Kevin Willis,23,82,2300,14.1,0.545,0.007,0.324,11.7,...,1.4,3.3,4.7,0.099,-1.1,0.0,-1.1,0.5,50,0
3,1986,Cliff Levingston,25,81,1945,14.7,0.572,0.002,0.439,11.0,...,2.6,2.9,5.5,0.136,0.4,1.4,1.7,1.8,50,0
4,1986,Tree Rollins,30,74,1781,11.0,0.537,0.003,0.259,8.1,...,0.5,3.0,3.4,0.093,-2.4,3.5,1.1,1.4,50,0


In [179]:
a[:3,:]

array([[1986, 26, 78, 3049, 23.3, 0.536, 0.037000000000000005,
        0.37200000000000005, 9.5, 12.9, 11.2, 11.7, 2.2, 0.9, 10.2, 32.9,
        nan, 7.0, 3.9, 10.8, 0.17, 3.5, 0.0, 3.5, 4.3, 50, 0],
       [1986, 26, 81, 2760, 13.0, 0.555, 0.018000000000000002, 0.153, 2.0,
        4.7, 3.4, 15.8, 1.4, 0.3, 10.8, 15.6, nan, 3.9, 2.0, 5.9,
        0.10300000000000001, 0.7, -0.1, 0.6, 1.8, 50, 0],
       [1986, 23, 82, 2300, 14.1, 0.545, 0.006999999999999999, 0.324, 11.7,
        22.0, 16.9, 2.8, 1.4, 1.1, 16.0, 19.6, nan, 1.4, 3.3, 4.7, 0.099,
        -1.1, 0.0, -1.1, 0.5, 50, 0]], dtype=object)

In [180]:
x = tf.placeholder(dtype=tf.float32, shape=([None, None, 27]), name='input')
y_true = tf.placeholder(dtype=tf.float32, shape=([None,1]), name='mvp')

class Helper():
    
    def __init__(self, data, test_data):
        self.i = 0
        self.all_train_batches = data
        self.test_images = test_data[:,:27]
        self.test_labels = test_data[:,27]
    
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b



In [181]:
ch = Helper(a,b)

out1 = normal_full_layer(x,5)
layer1 = tf.tanh(out1)

out2 = normal_full_layer(layer1, 3)
layer2 = tf.tanh(out2)

out3 = normal_full_layer(layer2, 1)
y_pred = tf.sigmoid(out3)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

init = tf.global_variables_initializer()

s = [0]*21
for year in temp:
    s[year-1986] += 1

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(5000):
        loopNum = i%20
        sess.run(train, feed_dict={x: batch[0], y_true: batch[1]})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            print(sess.run(acc,feed_dict={x:ch.test_images,y_true:ch.test_labels,hold_prob:1.0}))
            print('\n')


TypeError: unhashable type: 'slice'